# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import math
from scipy import stats
import xlsxwriter

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [121]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secret import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [122]:
import requests

# Define the API token and symbol
symbol = "AAPL"

# Construct the combined URL with the symbol and API token
api_url = f"https://cloud.iexapis.com/v1/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}"

# Make the API request
response = requests.get(api_url)

# Check if the response was successful (status code 200)
if response.status_code == 200:
    # Extract the JSON data from the response
    data = response.json()
    print(data)

{'companyName': 'Apple Inc', 'marketcap': 2605241628360, 'week52high': 175.37, 'week52low': 123.98, 'week52highSplitAdjustOnly': 176.15, 'week52lowSplitAdjustOnly': 124.17, 'week52change': -0.02483051796032876, 'sharesOutstanding': 15821946000, 'float': 0, 'avg10Volume': 53781228, 'avg30Volume': 64672410, 'day200MovingAvg': 145.64, 'day50MovingAvg': 154.95, 'employees': 164000, 'ttmEPS': 5.88, 'ttmDividendRate': 0.9170023129162208, 'dividendYield': 0.005569065425216937, 'nextDividendDate': '', 'exDividendDate': '2023-02-10', 'nextEarningsDate': '2023-05-04', 'peRatio': 27.37432230784588, 'beta': 1.2970212466416087, 'maxChangePercent': 62.930734586115854, 'year5ChangePercent': 3.108334684813932, 'year2ChangePercent': 0.27993557569506033, 'year1ChangePercent': -0.02483051796032876, 'ytdChangePercent': 0.269232933457384, 'month6ChangePercent': 0.18062332265232264, 'month3ChangePercent': 0.2722681699613048, 'month1ChangePercent': 0.0771243540262967, 'day30ChangePercent': 0.0934324988379042

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [123]:
data['year1ChangePercent']

-0.02483051796032876

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [124]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

for symbol_string in symbol_strings:
    print(symbol_string)
my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA
CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD
GIS,GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HFC,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,IEX,IFF,ILM

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [125]:
import warnings
warnings.filterwarnings("ignore")
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://cloud.iexapis.com/v1/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        if symbol == 'HFC' or symbol == 'VIAC' or symbol == 'WLTW' or symbol == 'DISCA':
            continue
        else:
            final_dataframe =  final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,138.03,0.030225,N/A
1,AAL,14.01,-0.155515,N/A
2,AAP,120.69,-0.450113,N/A
3,AAPL,164.66,-0.024831,N/A
4,ABBV,161.55,-0.041498,N/A
...,...,...,...,...
497,YUM,132.10,0.124586,N/A
498,ZBH,129.17,0.004828,N/A
499,ZBRA,300.26,-0.277579,N/A
500,ZION,29.39,-0.526055,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [126]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)

In [127]:
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,273,LB,79.92,2.281287,N/A
1,310,MNST,52.56,1.575839,N/A
2,193,FTI,13.43,0.708651,N/A
3,289,LW,106.86,0.631119,N/A
4,288,LVS,57.01,0.585814,N/A
5,463,VAR,177.07,0.571162,N/A
6,313,MPC,128.68,0.517043,N/A
7,490,WYNN,109.12,0.51073,N/A
8,271,KSU,293.59,0.506512,N/A
9,368,PHM,57.84,0.441664,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [128]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

portfolio_input()
print(portfolio_size)

Enter the value of your portfolio: 1000000


1000000


In [129]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,273,LB,79.92,2.281287,250
1,310,MNST,52.56,1.575839,380
2,193,FTI,13.43,0.708651,1489
3,289,LW,106.86,0.631119,187
4,288,LVS,57.01,0.585814,350
5,463,VAR,177.07,0.571162,112
6,313,MPC,128.68,0.517043,155
7,490,WYNN,109.12,0.51073,183
8,271,KSU,293.59,0.506512,68
9,368,PHM,57.84,0.441664,345


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [183]:
hqm_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                ]


hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://cloud.iexapis.com/v1/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        if symbol == 'HFC' or symbol == 'WLTW' or symbol == 'DISCA':
            continue
        else:
            hqm_dataframe = hqm_dataframe.append(
                                        pd.Series(
                                        [          symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   'N/A',
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month6ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month3ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month1ChangePercent'],
                                                   'N/A',
                                        ], 
                                        index = hqm_columns), 
                                        ignore_index = True
            )
        
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile
0,A,138.03,N/A,0.030225,N/A,0.08648,N/A,-0.063758,N/A,-0.018773,N/A
1,AAL,14.01,N/A,-0.155515,N/A,0.150246,N/A,-0.011989,N/A,-0.155515,N/A
2,AAP,120.69,N/A,-0.450113,N/A,-0.243955,N/A,-0.2164,N/A,-0.087134,N/A
3,AAPL,164.66,N/A,-0.024831,N/A,0.180623,N/A,0.272268,N/A,0.077124,N/A
4,ABBV,161.55,N/A,-0.041498,N/A,0.187267,N/A,-0.0208,N/A,0.07988,N/A
...,...,...,...,...,...,...,...,...,...,...,...
497,YUM,132.10,N/A,0.124586,N/A,0.218264,N/A,0.013892,N/A,0.041962,N/A
498,ZBH,129.17,N/A,0.004828,N/A,0.19769,N/A,0.029566,N/A,0.061574,N/A
499,ZBRA,300.26,N/A,-0.277579,N/A,0.125919,N/A,0.098164,N/A,-0.022177,N/A
500,ZION,29.39,N/A,-0.526055,N/A,-0.407152,N/A,-0.422155,N/A,-0.370394,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [140]:
from scipy.stats import percentileofscore as score

In [184]:
time_periods = [
    'One-Year',
    'Six-Month',
    'Three-Month',
    'One-Month'
]

# Replace NaN values with 0
hqm_dataframe.fillna(0, inplace=True)

for row in hqm_dataframe.index:
    for time_period in time_periods:
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        hqm_dataframe.loc[row, percentile_col] = score(hqm_dataframe[change_col], hqm_dataframe.loc[row, change_col])

# Print the entire DataFrame    
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile
0,A,138.03,N/A,0.030225,70.318725,0.086480,43.027888,-0.063758,28.087649,-0.018773,43.227092
1,AAL,14.01,N/A,-0.155515,29.880478,0.150246,58.167331,-0.011989,48.605578,-0.155515,6.175299
2,AAP,120.69,N/A,-0.450113,3.187251,-0.243955,3.386454,-0.216400,3.784861,-0.087134,18.12749
3,AAPL,164.66,N/A,-0.024831,56.175299,0.180623,66.135458,0.272268,97.011952,0.077124,87.450199
4,ABBV,161.55,N/A,-0.041498,53.187251,0.187267,68.12749,-0.020800,44.621514,0.079880,89.243028
...,...,...,...,...,...,...,...,...,...,...,...
497,YUM,132.10,N/A,0.124586,82.868526,0.218264,74.900398,0.013892,60.358566,0.041962,72.50996
498,ZBH,129.17,N/A,0.004828,64.741036,0.197690,70.517928,0.029566,66.932271,0.061574,80.876494
499,ZBRA,300.26,N/A,-0.277579,12.151394,0.125919,52.589641,0.098164,86.055777,-0.022177,41.434263
500,ZION,29.39,N/A,-0.526055,1.992032,-0.407152,0.796813,-0.422155,0.59761,-0.370394,0.59761


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [185]:
from statistics import mean
hqm_dataframe['HQM Score'] = 0
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,138.03,N/A,0.030225,70.318725,0.086480,43.027888,-0.063758,28.087649,-0.018773,43.227092,0
1,AAL,14.01,N/A,-0.155515,29.880478,0.150246,58.167331,-0.011989,48.605578,-0.155515,6.175299,0
2,AAP,120.69,N/A,-0.450113,3.187251,-0.243955,3.386454,-0.216400,3.784861,-0.087134,18.12749,0
3,AAPL,164.66,N/A,-0.024831,56.175299,0.180623,66.135458,0.272268,97.011952,0.077124,87.450199,0
4,ABBV,161.55,N/A,-0.041498,53.187251,0.187267,68.12749,-0.020800,44.621514,0.079880,89.243028,0
...,...,...,...,...,...,...,...,...,...,...,...,...
497,YUM,132.10,N/A,0.124586,82.868526,0.218264,74.900398,0.013892,60.358566,0.041962,72.50996,0
498,ZBH,129.17,N/A,0.004828,64.741036,0.197690,70.517928,0.029566,66.932271,0.061574,80.876494,0
499,ZBRA,300.26,N/A,-0.277579,12.151394,0.125919,52.589641,0.098164,86.055777,-0.022177,41.434263,0
500,ZION,29.39,N/A,-0.526055,1.992032,-0.407152,0.796813,-0.422155,0.59761,-0.370394,0.59761,0


In [186]:
for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [187]:
hqm_dataframe.sort_values('HQM Score', ascending = False, inplace = True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(drop = True,inplace = True)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,MNST,52.56,N/A,1.575839,99.800797,1.328497,100.0,1.052524,100.0,1.038790,100.0,99.950199
1,NVDA,270.37,N/A,0.170564,87.450199,1.239857,99.800797,0.819879,99.800797,0.118109,97.609562,96.165339
2,LB,79.92,N/A,2.281287,100.0,0.815831,99.203187,0.215421,95.816733,0.079271,88.645418,95.916335
3,GE,93.60,N/A,0.331033,96.414343,0.845673,99.601594,0.302290,97.609562,0.076109,86.85259,95.119522
4,FDX,232.10,N/A,0.178290,88.047809,0.521684,97.808765,0.256346,96.613546,0.113828,96.812749,94.820717
5,ANET,159.85,N/A,0.219950,91.035857,0.373164,93.227092,0.421521,99.003984,0.077156,87.649402,92.729084
6,PHM,57.84,N/A,0.441664,98.207171,0.452073,96.613546,0.200700,95.219124,0.059467,79.482072,92.380478
7,ORCL,95.92,N/A,0.200982,89.641434,0.529066,98.207171,0.123975,89.442231,0.083964,90.239044,91.882470
8,COTY,11.64,N/A,0.355064,97.011952,0.838863,99.40239,0.265217,96.812749,0.028269,66.533865,89.940239
9,COG,22.25,N/A,0.340483,96.812749,0.180873,66.334661,0.255629,96.414343,0.249298,99.40239,89.741036


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [159]:
portfolio_input()

Enter the value of your portfolio: 1000000


In [188]:
position_size = float(portfolio_size) / len(hqm_dataframe.index)
for i in range(0, len(hqm_dataframe['Ticker'])):
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe['Price'][i])
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,MNST,52.56,380,1.575839,99.800797,1.328497,100.0,1.052524,100.0,1.038790,100.0,99.950199
1,NVDA,270.37,73,0.170564,87.450199,1.239857,99.800797,0.819879,99.800797,0.118109,97.609562,96.165339
2,LB,79.92,250,2.281287,100.0,0.815831,99.203187,0.215421,95.816733,0.079271,88.645418,95.916335
3,GE,93.60,213,0.331033,96.414343,0.845673,99.601594,0.302290,97.609562,0.076109,86.85259,95.119522
4,FDX,232.10,86,0.178290,88.047809,0.521684,97.808765,0.256346,96.613546,0.113828,96.812749,94.820717
5,ANET,159.85,125,0.219950,91.035857,0.373164,93.227092,0.421521,99.003984,0.077156,87.649402,92.729084
6,PHM,57.84,345,0.441664,98.207171,0.452073,96.613546,0.200700,95.219124,0.059467,79.482072,92.380478
7,ORCL,95.92,208,0.200982,89.641434,0.529066,98.207171,0.123975,89.442231,0.083964,90.239044,91.882470
8,COTY,11.64,1718,0.355064,97.011952,0.838863,99.40239,0.265217,96.812749,0.028269,66.533865,89.940239
9,COG,22.25,898,0.340483,96.812749,0.180873,66.334661,0.255629,96.414343,0.249298,99.40239,89.741036


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [190]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [191]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [192]:
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['One-Year Price Return', percent_template],
                    'E': ['One-Year Return Percentile', percent_template],
                    'F': ['Six-Month Price Return', percent_template],
                    'G': ['Six-Month Return Percentile', percent_template],
                    'H': ['Three-Month Price Return', percent_template],
                    'I': ['Three-Month Return Percentile', percent_template],
                    'J': ['One-Month Price Return', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['HQM Score', integer_template]
                    }

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [193]:
writer.save()